In [4]:
import matplotlib.pyplot as plt
import numpy as np
from lattice import Lattice2D, BrickwallLatticeGeometry, SimulationParameters

In [11]:
omega = 2 * np.pi / 4

geometry = BrickwallLatticeGeometry((7, 14))

# Convergence Testing

Test RK4 solver with different step sizes; compare sparse and non-sparse hamiltonian.

In [ ]:
import time

# Define step sizes to test (from smallest to largest)
h_values = [1e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 1e-1]

# Function to run simulation and return results and timing
def run_simulation(h, use_sparse=True):
    start_time = time.time()
    
    params_h = SimulationParameters(
        t_hop=1.0,
        E_amplitude=lambda t: np.sin(omega * t),
        E_direction=np.array([0, -1]),
        h=h,
        T=10,
        charge=1,
        initial_occupation=0.5,
    )
    
    lattice = Lattice2D(geometry, params_h)
    lattice.evolve(solver="rk4", use_sparse=use_sparse, sample_every=0)
    final_state = lattice.states[-1].density.diagonal().real
    
    elapsed_time = time.time() - start_time
    return final_state, elapsed_time

In [16]:
results_sparse = []
times_sparse = []
results_dense = []
times_dense = []

for h in h_values:
    print(f"Running with h={h}")
    
    # print("  - Sparse matrix...")
    final_state, elapsed = run_simulation(h, use_sparse=True)
    results_sparse.append(final_state)
    times_sparse.append(elapsed)
    print(f"Completed in {elapsed:.2f} seconds")
    
    # print("  - Dense matrix...")
    # final_state, elapsed = run_simulation(h, use_sparse=False)
    # results_dense.append(final_state)
    # times_dense.append(elapsed)
    # print(f"    Completed in {elapsed:.2f} seconds")

In [27]:
# Calculate standard deviations from the reference solution (smallest h)
reference_sparse = results_sparse[0]
# reference_dense = results_dense[0]

deviations_sparse = [np.std(result - reference_sparse) for result in results_sparse]
# deviations_dense = [np.std(result - reference_dense) for result in results_dense]

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Plot standard deviations vs h
ax1.loglog(h_values, deviations_sparse, 'o-', label='Sparse')
# ax1.set_xscale('log')
# ax1.loglog(h_values, deviations_dense, 's-', label='Dense')
ax1.set_xlabel('Step size (h)')
ax1.set_ylabel('Standard deviation from reference')
ax1.set_title('Convergence Analysis')
ax1.legend()
ax1.grid(True)

# Plot computation time vs h
ax2.loglog(h_values, times_sparse, 'o-', label='Sparse')
# ax2.loglog(h_values, times_dense, 's-', label='Dense')
ax2.set_xlabel('Step size (h)')
ax2.set_ylabel('Computation time (s)')
ax2.set_title('Performance Analysis')
ax2.legend()
ax2.grid(True)

# Plot efficiency: deviation vs time
ax3.loglog(times_sparse, deviations_sparse, 'o-', label='Sparse')
# ax3.loglog(times_dense, deviations_dense, 's-', label='Dense')
ax3.set_xlabel('Computation time (s)')
ax3.set_ylabel('Standard deviation from reference')
ax3.set_title('Efficiency Analysis')
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.show()

## Analysis and Optimal Step Size

We can determine an optimal step size by finding where the deviation starts to grow significantly. This represents the best balance between accuracy and performance.

In [ ]:
# Find optimal h based on deviation threshold
threshold = 1e-7

print("Sparse Matrix Results:")
print("-" * 60)
print(f"{'Step size (h)':<15} {'Deviation':<15} {'Time (s)':<15} {'Optimal?':<15}")
print("-" * 60)

for h, dev, t in zip(h_values, deviations_sparse, times_sparse):
    optimal = "Yes" if dev < threshold else "No"
    print(f"{h:<15.6f} {dev:<15.6e} {t:<15.3f} {optimal:<15}")

# print("\nDense Matrix Results:")
# print("-" * 60)
# print(f"{'Step size (h)':<15} {'Deviation':<15} {'Time (s)':<15} {'Optimal?':<15}")
# print("-" * 60)

# for h, dev, t in zip(h_values, deviations_dense, times_dense):
#     optimal = "Yes" if dev < threshold else "No"
#     print(f"{h:<15.6f} {dev:<15.6e} {t:<15.3f} {optimal:<15}")